In [325]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [326]:
class ExoplanetDataset(Dataset):
    """Exoplanet dataset."""

    def __init__(self, X, y, standardize=True, totensor=True, moving_average=1):
        self.X = X
        self.y = y
        self.standardize = standardize
        self.totensor = totensor
        self.moving_average = moving_average

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]
        x = np.convolve(x, np.ones((self.moving_average,))/self.moving_average, mode='valid')
        
        if self.standardize:
            x = (x - np.mean(x)) / (np.std(x) + 0.01)
        
        if self.totensor:
            x = torch.tensor(x, dtype=torch.float).unsqueeze(0)

        sample = (int(self.y[idx]), x)
        return sample

In [332]:
def remove_nan(data):
    return data[~np.isnan(data).any(axis=1)]


def split_train_test(data, ratio=0.7, shuffle=True):
    np.random.shuffle(data)
    
    length = len(data)
    data_train = data[0 : int(ratio * length)]
    data_test = data[int(ratio * length) : ]
    return data_train, data_test

def split_X_y(data):
    X = data[:, 0:-1]
    y = data[:, -1]- 1
    return X, y

In [328]:
### Import data ###
data = np.genfromtxt("../../data/kepler-labelled-time-series-data/exoTrain_AUG_FILT.csv", delimiter=',')
data = remove_nan(data)

In [343]:
### Setup for training ###
data_train, data_test = split_train_test(data)
X_train, y_train = split_X_y(data_train)
X_test, y_test = split_X_y(data_test)

print("X_train shape: {};".format(X_train.shape))
print("X_test shape: {};".format(X_test.shape))



dataset_train = ExoplanetDataset(X_train, y_train, moving_average=100)
dataset_test = ExoplanetDataset(X_test, y_test, moving_average=100)

cuda_available = torch.cuda.is_available()

device = torch.device("cuda" if cuda_available else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_available else {}
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True, **kwargs)

X_train shape: (7070, 3197);
X_test shape: (3030, 3197);


In [348]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 20, kernel_size=5, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.conv2 = nn.Conv1d(20, 50, kernel_size=5, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=3)
        self.conv3 = nn.Conv1d(50, 100, kernel_size=3, stride=1)
        self.maxpool3 = nn.MaxPool1d(kernel_size=3)
        
        self.fc1 = nn.Linear(100 * 204, 500)
        self.fc2 = nn.Linear(500, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool3(x)
        x = x.view(-1, 100 * 204)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    

### Model initialization ###
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [349]:
def train(model, device, train_loader, criterion, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (target, data) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


### Training ###         
for epoch in range(10):
    train(model, device, train_loader, criterion, optimizer, epoch, 100)

Train Epoch: 0 [0/7070 (0%)]	Loss: 0.698090
Train Epoch: 0 [3200/7070 (45%)]	Loss: 0.634418
Train Epoch: 0 [6400/7070 (90%)]	Loss: 0.474656
Train Epoch: 1 [0/7070 (0%)]	Loss: 0.468168
Train Epoch: 1 [3200/7070 (45%)]	Loss: 0.271469
Train Epoch: 1 [6400/7070 (90%)]	Loss: 0.159227
Train Epoch: 2 [0/7070 (0%)]	Loss: 0.175148
Train Epoch: 2 [3200/7070 (45%)]	Loss: 0.110718
Train Epoch: 2 [6400/7070 (90%)]	Loss: 0.085273
Train Epoch: 3 [0/7070 (0%)]	Loss: 0.049819
Train Epoch: 3 [3200/7070 (45%)]	Loss: 0.018452
Train Epoch: 3 [6400/7070 (90%)]	Loss: 0.008251
Train Epoch: 4 [0/7070 (0%)]	Loss: 0.028084
Train Epoch: 4 [3200/7070 (45%)]	Loss: 0.004529
Train Epoch: 4 [6400/7070 (90%)]	Loss: 0.004936
Train Epoch: 5 [0/7070 (0%)]	Loss: 0.008359
Train Epoch: 5 [3200/7070 (45%)]	Loss: 0.007792
Train Epoch: 5 [6400/7070 (90%)]	Loss: 0.009328
Train Epoch: 6 [0/7070 (0%)]	Loss: 0.008606
Train Epoch: 6 [3200/7070 (45%)]	Loss: 0.001415
Train Epoch: 6 [6400/7070 (90%)]	Loss: 0.003473
Train Epoch: 7 [0/70

In [350]:
def test(model, device, dataset_test):
    accuracy = 0
    for i in range(len(dataset_test)):
        output = np.argmax(model(dataset[i][1].view(-1, 1, 3098)).detach().numpy())
        target = dataset[i][0]
        accuracy += (output == target)
        
    accuracy /= len(dataset_test)
    print("Accuracy: {}% over {} data points.".format(accuracy, len(dataset_test)))
    

### Test ###
test(model, device, dataset_test)

Accuracy: 0.997029702970297% over 3030 data points.
